In [ ]:
import os
import sys

# Clone or pull part
repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/kaggle/working/6D_pose"   #Modify here for kaggle
branch = "pose_rgbd"

# Clone if missing
if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url} to {repo_dir}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    # %cd ..
    print(f"Updated {repo_url} to {repo_dir}")

# Add repository to Python path
if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)


In [ ]:
%cd ..
!gdown --fuzzy https://drive.google.com/file/d/1zNthSyiBdPUfn7BmUKPbKoGgQdG1vGnS/view?usp=drive_link -O Linemod_preprocessed.zip
!unzip Linemod_preprocessed.zip
%cd 6D_pose

In [ ]:
from google.colab import drive
from utils.load_data import mount_drive

# Mounting part
mount_drive()

In [ ]:
%mv Linemod_preprocessed working/

In [ ]:
# dataset_root = "/content/drive/MyDrive/Linemod_preprocessed" #Modify here for kaggle
# dataset_root = "../../Linemod_preprocessed"
# dataset_root = "/content/Linemod_preprocessed"
dataset_root = "/kaggle/working/Linemod_preprocessed"

print("\n✅ Setup complete!")
print(f"📁 Dataset path: {dataset_root}")


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%mv Linemod_preprocessed ./working

In [ ]:
!pip install plyfile

In [ ]:
import importlib

importlib.reload(sys.modules['src.pose_rgbd.dataset'])

In [ ]:
from src.pose_rgbd.loss import GeodesicLoss
from src.pose_rgbd.dataset import LineModPoseDepthDataset

train_dataset = LineModPoseDepthDataset(
    root_dir=dataset_root,
    split="train"
)

test_dataset = LineModPoseDepthDataset(
    root_dir=dataset_root,
    split="test"
)


In [ ]:
sample = train_dataset[0]

print(f"Sample keys: {sample.keys()}")
print(f"Depth shape: {sample['depth'].shape}")
print(f"RGB shape: {sample['image'].shape}")

In [ ]:
import torch
from src.pose_rgbd.model import RGBDRotationModel


# Configurazione del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
from utils.linemod_config import get_linemod_config
import numpy as np
import torch

# dataset_root = "../../Linemod_preprocessed_small"  # Adjust path as needed
linemod_config = get_linemod_config(dataset_root)

all_model_points = []
NUM_POINTS = 1000  # Number of points to sample from each model
VALID_OBJ_IDS = [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15] 
for obj_id in VALID_OBJ_IDS:
    model_points = linemod_config.get_model_3d(obj_id, unit='m')  # (N, 3)
    if model_points.shape[0] >= NUM_POINTS:
        choice = np.random.choice(model_points.shape[0], NUM_POINTS, replace=False)
    else:
        choice = np.random.choice(model_points.shape[0], NUM_POINTS, replace=True)
    model_points = model_points[choice, :]
    all_model_points.append(torch.tensor(model_points, dtype=torch.float32))
all_model_points = torch.stack(all_model_points, dim=0)  # (Num_Classes, NUM_POINTS, 3)
all_model_points = all_model_points.to(device)

max_obj_id = max(VALID_OBJ_IDS)

# Create a lookup table: obj_id -> index
obj_id_to_idx = torch.full((max_obj_id + 1,), -1, dtype=torch.long, device=device)
for idx, obj_id in enumerate(VALID_OBJ_IDS):
    obj_id_to_idx[obj_id] = idx

In [ ]:
all_model_points.shape

In [ ]:
from src.pose_rgbd.loss import MultiObjectPointMatchingLoss
import torch.nn as nn

model = RGBDRotationModel(pretrained=True)

if torch.cuda.device_count() > 1:
    print(f"🔥 Using {torch.cuda.device_count()} GPU!")
    model = nn.DataParallel(model)

model = model.to(device)

backbone_params = [p for n, p in model.named_parameters() if 'backbone' in n]
head_params = [p for n, p in model.named_parameters() if 'rot_head' in n]

# Loss function and optimizer
criterion = MultiObjectPointMatchingLoss(all_model_points)
optimizer = torch.optim.AdamW([
    {
        'params': backbone_params, 
        'lr': 3e-5  # Low: preserves knowledge, adapts only depth
    },
    {
        'params': head_params,     
        'lr': 1e-3  # High: learn to predict quaternions from scratch
    }
], weight_decay=1e-4) # Global regularization

# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',       # We want to minimize the validation loss
    factor=0.5,       # Halve the learning rate when triggered
    patience=3,       # Wait 3 epochs of stagnation (small dataset -> low patience)
    verbose=True,
    min_lr=1e-7
)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 32  #double GPU

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
from tqdm import tqdm
import os
import datetime
from itertools import islice
from datetime import datetime

# Load best model

# Ciclo di training
num_epochs = 15
best_test_loss = float('inf')
# checkpoint_dir = "checkpoints"
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# checkpoint_dir = f'/content/drive/MyDrive/runs/POINT_MATCH_{timestamp}' # modify here for kaggle
checkpoint_dir = f'/kaggle/working/POINT_MATCH_{timestamp}'
os.makedirs(checkpoint_dir, exist_ok=True)
# val_batches_limit = 50

# Track losses for plotting
train_losses = []
test_losses = []

batch_size = 32

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    train_pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] - Training")
    for batch in train_pbar:
        rgb = batch['image'].to(device)  # RGB image (B, 3, H, W)
        depth = batch['depth'].to(device)  # Depth (B, 1, H, W)
        rotations = batch['rotation'].to(device)  # GT quaternion (B, 4)
        obj_ids = batch['object_id'].to(device)  # Object IDs (B,)
        obj_ids = obj_id_to_idx[obj_ids]  # Map to indices (B,)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(rgb, depth)

        # Calcolo della loss
        loss = criterion(outputs, rotations, obj_ids)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        train_pbar.set_postfix({'loss': epoch_loss / (train_pbar.n + 1)})


    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    test_loss = 0.0

    val_pbar = tqdm(test_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] - Validating")
    count_batches = 0
    with torch.no_grad():
        for batch in val_pbar:
            rgb = batch['image'].to(device)
            depth = batch['depth'].to(device)
            rotations = batch['rotation'].to(device)
            obj_ids = batch['object_id'].to(device)  # Object IDs (B,)
            obj_ids = obj_id_to_idx[obj_ids].to(device)  # Map to indices (B,)

            outputs = model(rgb, depth)
            loss = criterion(outputs, rotations, obj_ids)
            test_loss += loss.item()
            val_pbar.set_postfix({'loss': test_loss / (val_pbar.n + 1)})
            count_batches+=1

    avg_test_loss = test_loss / len(test_loader)
    test_losses.append(avg_test_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Test Loss: {avg_test_loss:.4f}")

    # Step the scheduler
    scheduler.step(avg_test_loss)

    # Save checkpoint if test loss improved
    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        checkpoint_path = os.path.join(checkpoint_dir, "best_model.pth")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'test_loss': avg_test_loss,
        }, checkpoint_path)
        print(f"✅ Checkpoint saved! Best Test Loss: {best_test_loss:.4f}")
    else:
        print(f"⚠️  No improvement. Best Test Loss: {best_test_loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Create plots directory
# plots_dir = "plots"
plots_dir = checkpoint_dir
os.makedirs(plots_dir, exist_ok=True)

# Plot 1: Training vs Test Loss
plt.figure(figsize=(10, 6))
epochs_range = range(1, len(test_losses)+1)
plt.plot(range(1, len(train_losses)+1), train_losses, 'b-o', label='Training Loss', linewidth=2, markersize=6)
plt.plot(range(1, len(test_losses)+1), test_losses, 'r-s', label='Test Loss', linewidth=2, markersize=6)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training vs Test Loss', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
loss_plot_path = os.path.join(plots_dir, "loss_comparison.png")
plt.savefig(loss_plot_path, dpi=300, bbox_inches='tight')
print(f"✅ Plot saved: {loss_plot_path}")
plt.show()

# Plot 2: Only Training Loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses)+1), train_losses, 'b-o', linewidth=2, markersize=6)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Training Loss', fontsize=12)
plt.title('Training Loss Over Epochs', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
train_loss_path = os.path.join(plots_dir, "training_loss.png")
plt.savefig(train_loss_path, dpi=300, bbox_inches='tight')
print(f"✅ Plot saved: {train_loss_path}")
plt.show()

# Plot 3: Only Test Loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(test_losses)+1), test_losses, 'r-s', linewidth=2, markersize=6)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Test Loss', fontsize=12)
plt.title('Test Loss Over Epochs', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.axhline(y=best_test_loss, color='g', linestyle='--', label=f'Best: {best_test_loss:.4f}', linewidth=2)
plt.legend(fontsize=11)
plt.tight_layout()
test_loss_path = os.path.join(plots_dir, "test_loss.png")
plt.savefig(test_loss_path, dpi=300, bbox_inches='tight')
print(f"✅ Plot saved: {test_loss_path}")
plt.show()

print(f"\n✅ All plots saved in '{plots_dir}' directory!")

In [ ]:
# Save losses
import pickle


losses_dict = {
    'train_losses': train_losses,
    'test_losses': test_losses
}

losses_path = os.path.join(checkpoint_dir, "losses.pkl")
with open(losses_path, 'wb') as f:
    pickle.dump(losses_dict, f)


# Visualize samples

In [ ]:
import random
import cv2
from utils.projection_utils import setup_projection_utils, visualize_pose_comparison, get_image
import matplotlib.pyplot as plt

# Setup projection utils
setup_projection_utils(dataset_root)

# Load best model
best_checkpoint_path = os.path.join(checkpoint_dir, "best_model.pth")
checkpoint = torch.load(best_checkpoint_path)

state_dict = checkpoint['model_state_dict']

# 2. Crea un nuovo dizionario senza il prefisso 'module.'
from collections import OrderedDict
new_state_dict = OrderedDict()

for k, v in state_dict.items():
    # Rimuovi 'module.' se presente all'inizio della chiave
    name = k[7:] if k.startswith('module.') else k 
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
print(f"✅ Loaded best model from epoch {checkpoint['epoch']} with test loss: {checkpoint['test_loss']:.4f}")

# Select a random sample from test dataset
random_idx = random.randint(0, len(test_dataset) - 1)
sample = test_dataset[random_idx]

print(f"\n📷 Visualizing sample {random_idx}:")
print(f"   Object ID: {sample['object_id']}")
print(f"   Image ID: {sample['img_id']}")

# Get the original image
img_path = sample['img_path']
image_rgb = cv2.imread(str(img_path))
# image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)

# Prepare input for model
rgb = sample['image'].unsqueeze(0).to(device)  # Add batch dimension
depth = sample['depth'].unsqueeze(0).to(device)  # Add batch and channel dimensions

# Get model prediction
model.eval()
with torch.no_grad():
    pred_rotation = model(rgb, depth)[0].cpu().numpy()

# Get ground truth
gt_rotation = sample['rotation'].numpy()
gt_translation = sample['translation'].numpy()

# Get camera intrinsics
cam_K = sample['cam_K'].numpy()

print(f"\n📊 Ground Truth vs Prediction:")
print(f"   GT Rotation: {gt_rotation}")
print(f"   Pred Rotation: {pred_rotation}")
print(f"   GT Translation: {gt_translation}")

# Visualize pose comparison
fig, ax = plt.subplots(1, 1, figsize=(14, 8))
img_vis = visualize_pose_comparison(
    image_rgb,
    object_id=sample['object_id'],
    cam_K=cam_K,
    gt_rotation=gt_rotation,
    gt_translation=gt_translation,
    pred_rotation=pred_rotation,
    pred_translation=gt_translation  # Using GT translation for now
)

# Convert BGR to RGB for matplotlib
img_vis_rgb = cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB)
ax.imshow(img_vis_rgb)
ax.axis('off')
ax.set_title(f"Pose Visualization - Object {sample['object_id']}, Image {sample['img_id']}", fontsize=14, fontweight='bold')
plt.tight_layout()
# plt.savefig(os.path.join(plots_dir, f"pose_visualization_sample_{random_idx}.png"), dpi=150, bbox_inches='tight')
print(f"\n✅ Visualization saved!")
plt.show()

# Evaluation

In [ ]:
from src.pose_rgbd.evaluate import evaluate_RGBD
MODEL_PATH = "./RGBD_run/best_model.pth"            # Path to the trained model
DATASET_ROOT = "../../Linemod_preprocessed"  # Path to the Linemod dataset
OUTPUT_PATH = "./RGBD_run/linemod_evaluation_report.csv"  # Path to save the evaluation report

df = evaluate_RGBD(
    model_path=MODEL_PATH,
    dataset_root=DATASET_ROOT,
    output_path=OUTPUT_PATH
)

df